In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from scipy.stats import linregress
from scipy import stats
import gmaps

import json

# Import API key
from api_keys import g_key

# Incorporated citipy to determine city based on latitude and longitude
from citipy import citipy

# Input File (CSV)
input_data_file = "output_data/cities.csv"


In [2]:
weather_df = pd.read_csv(input_data_file)

weather_df.head()

,Unnamed: 0,Latitude,Longitude,Temperature,Max Temp,Humidity,City,Wind Speed,Cloudiness
0,0,9.0167,37.4500,51.08,51.08,99,Gēdo,2.15,94
1,0,69.4865,88.3972,65.21,65.21,47,Talnakh,1.23,99
2,0,55.7435,-97.8558,66.36,66.36,59,Thompson,2.30,20
3,0,-46.6000,168.3333,50.81,50.81,74,Bluff,10.69,99
4,0,-53.1500,-70.9167,37.51,37.51,87,Punta Arenas,10.36,40


In [3]:
# Configure gmaps with API key.
gmaps.configure(api_key=g_key)

# Convert Humidity to float and store
# Also, handle NaN values
weather_df = weather_df.dropna()
humidity = weather_df["Humidity"].astype(float)

# Store 'Latitude' and 'Longitude' into  locations. 
locations = weather_df[["Latitude", "Longitude"]].astype(float)

# Create a humidity Heatmap layer
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=max(weather_df["Humidity"]),
                                 point_radius = 2)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

In [4]:
# Narrow down the DataFrame to find your ideal weather condition.
filtered_weather_df = weather_df

# Drop any rows that don't contain all three conditions. Want to be sure the weather is ideal.

# A max temperature lower than 80 degrees but higher than 70.
filtered_weather_df = filtered_weather_df.loc[(filtered_weather_df["Max Temp"] < 80) & (filtered_weather_df["Max Temp"] > 70)]

# Wind speed less than 10 mph.
filtered_weather_df = filtered_weather_df.loc[filtered_weather_df["Wind Speed"] < 10]

# Zero cloudiness.
filtered_weather_df = filtered_weather_df.loc[filtered_weather_df["Cloudiness"] == 0]

# Drop any rows with null values
filtered_weather_df = filtered_weather_df.dropna()

filtered_weather_df

,Unnamed: 0,Latitude,Longitude,Temperature,Max Temp,Humidity,City,Wind Speed,Cloudiness
49,0,31.6100,34.7642,69.55,75.88,100,Kiryat Gat,1.81,0
78,0,32.6667,-17.1000,70.43,72.01,76,Ponta do Sol,1.48,0
100,0,28.4190,30.7792,76.48,76.48,64,Maţāy,9.51,0
123,0,47.8667,88.1167,73.53,73.53,33,Altay,5.57,0
205,0,29.9737,32.5263,73.31,73.31,77,Suez,6.80,0
222,0,31.6315,-8.0083,77.07,77.07,53,Marrakesh,2.30,0
244,0,32.5341,20.5791,78.31,78.31,77,Tūkrah,4.29,0
276,0,-21.3393,55.4781,64.81,70.92,71,Saint-Pierre,5.75,0
280,0,45.2986,35.7771,73.80,73.80,78,Lenine,3.36,0
374,0,37.0397,45.0983,72.16,72.16,27,Oshnavīyeh,2.80,0


In [11]:
hotel_df = filtered_weather_df

# params dictionary to update each iteration
params = {
    "radius": 5000000000,
    "types": "lodging",
    "key": g_key
}


for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Latitude"]
    lng = row["Longitude"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {row['City']}.")
    response = requests.get(base_url, params=params).json()
    # print(json.dumps(response, indent=4, sort_keys=True))
    
    # extract results
    results = response['results']
    
    print(results)
    
    try:
        print(f"Closest hotel is {results[0]['name']}.")
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")

Retrieving Results for Index 49: Kiryat Gat.
[]
Missing field/result... skipping.
------------
Retrieving Results for Index 78: Ponta do Sol.
[]
Missing field/result... skipping.
------------
Retrieving Results for Index 100: Maţāy.
[]
Missing field/result... skipping.
------------
Retrieving Results for Index 123: Altay.
[]
Missing field/result... skipping.
------------
Retrieving Results for Index 205: Suez.
[]
Missing field/result... skipping.
------------
Retrieving Results for Index 222: Marrakesh.
[]
Missing field/result... skipping.
------------
Retrieving Results for Index 244: Tūkrah.
[]
Missing field/result... skipping.
------------
Retrieving Results for Index 276: Saint-Pierre.
[]
Missing field/result... skipping.
------------
Retrieving Results for Index 280: Lenine.
[]
Missing field/result... skipping.
------------
Retrieving Results for Index 374: Oshnavīyeh.
[]
Missing field/result... skipping.
------------
Retrieving Results for Index 457: Quelimane.
[]
Missing field/r

In [8]:
hotel_df

,Unnamed: 0,Latitude,Longitude,Temperature,Max Temp,Humidity,City,Wind Speed,Cloudiness
49,0,31.6100,34.7642,69.55,75.88,100,Kiryat Gat,1.81,0
78,0,32.6667,-17.1000,70.43,72.01,76,Ponta do Sol,1.48,0
100,0,28.4190,30.7792,76.48,76.48,64,Maţāy,9.51,0
123,0,47.8667,88.1167,73.53,73.53,33,Altay,5.57,0
205,0,29.9737,32.5263,73.31,73.31,77,Suez,6.80,0
222,0,31.6315,-8.0083,77.07,77.07,53,Marrakesh,2.30,0
244,0,32.5341,20.5791,78.31,78.31,77,Tūkrah,4.29,0
276,0,-21.3393,55.4781,64.81,70.92,71,Saint-Pierre,5.75,0
280,0,45.2986,35.7771,73.80,73.80,78,Lenine,3.36,0
374,0,37.0397,45.0983,72.16,72.16,27,Oshnavīyeh,2.80,0
